## Imports

In [ ]:
import json, os, sys
import pickle, csv
import time
import datetime
from collections import OrderedDict
import itertools
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

import librosa
from sklearn.neighbors import KernelDensity
from IPython.display import Audio as ipy_audio

In [ ]:
from quicktranscribe import tonic, pitch, wave, kde

## Data

In [ ]:
data_dir = "/Users/neerajaabhyankar/Repos/icm-shruti-analysis/data-dunya-hindustani/"
track = "Omkar Dadarkar - Raag Bhoopali"

Metadata, Tonic, Pitch, Audio

In [ ]:
# metadata = tonic.read_metadata(data_dir + track + ".json")
ctonic = tonic.read_tonic(data_dir + track + ".ctonic.txt")
pitch_annotations, aps = pitch.read_pitch(data_dir + track + ".pitch.txt")

In [ ]:
# y, sr = wave.get_audio(data_dir + track + ".mp3")

In [ ]:
# Find a representative sample
start=9*60+45
end=10*60
y_stereo, sr = wave.read_audio_section(data_dir + track + ".mp3", start, end)
y_sample = librosa.to_mono(y_stereo.T)
ipy_audio(data=y_sample, rate=sr)

In [ ]:
# code from below repurposed into a single function
kde_sample = kde.extract(y_sample, sr=sr, tonic=ctonic)

plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="green")
plt.xlabel("relative note index")
plt.ylabel("normalized duration")

## Librosa Pitch Detection with `pyin`

In [ ]:
# params
fpd_bin_width = 0.05  # of a midi note
frame_length = 2048
frame_hop_length = frame_length // 4

In [ ]:
# pitch tracking
f0, voiced_flag, voiced_probs = librosa.pyin(
    y_sample,
    fmin=ctonic/2, fmax=ctonic*4,
    frame_length=frame_length
)

# convert frequency to MIDI for easier binning
# needed since midi is in log scale already
f0_midi = librosa.hz_to_midi(f0)

# subtract tonic + fold into octave
f0_relative = (f0_midi-librosa.hz_to_midi(ctonic)) % 12
f0_relative = f0_relative[~np.isnan(f0_relative)]

# get a duration-binned histogram
fpd_bins = np.arange(0, 12.0001, fpd_bin_width)
pitch_distribution, _ = np.histogram(f0_relative, bins=fpd_bins)
time_per_frame = librosa.frames_to_time(1, sr=sr, hop_length=frame_hop_length)
duration_per_bin = pitch_distribution * time_per_frame

In [ ]:
# plot
plt.figure(figsize=(10, 6))
plt.bar(fpd_bins[:-1], duration_per_bin, width=fpd_bin_width)
plt.xlabel("relative note index (5-cent granularity)")
plt.ylabel("duration (s)")
plt.title("Finegrained Pitch Distribution")
plt.show()

## Kernel Smoothed PD

In [ ]:
kde_granularity = 240
kde_bandwidth = 0.1
supp = np.linspace(0, 12, kde_granularity).reshape(-1, 1)

In [ ]:
np.random.seed(1)
kde = KernelDensity(kernel="gaussian", bandwidth=kde_bandwidth).fit(f0_relative.reshape(-1, 1))
logkde = kde.score_samples(supp)
kde_sample = np.exp(logkde)
plt.plot(supp, duration_per_bin, alpha=0.5)
plt.plot(supp, kde_sample, color="green")
plt.show()

In [ ]:
# # alternatively
# from scipy import stats
# kde = stats.gaussian_kde(f0_relative, bw_method=0.01)
# x = np.linspace(0, 12, kde_granularity)
# p = kde(x)

## Librosa Pitch Detection with `piptrack`

In [ ]:
def detect_pitch(y, sr):
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr, fmin=110, fmax=320)
    # get indexes of the maximum value in each time slice
    max_indexes = np.argmax(magnitudes, axis=0)
    # get the pitches of the max indexes per time slice
    pitches = pitches[max_indexes, range(magnitudes.shape[1])]
    return pitches

In [ ]:
p_sample = detect_pitch(y_sample, sr)
ar = len(p_sample)/(end-start)

In [ ]:
plt.plot(p_sample)

In [ ]:
np_sample = np.array(p_sample/ctonic)
nz_annotations = np_sample[np.where(np_sample[:] != 0)]
h = np.histogram(nz_annotations, bins=700)
counts = h[0]
fro = (h[1][:-1] + h[1][1:]) / 2

plt.figure()
fig = px.line(
    pd.DataFrame({"fratio": fro, "count": counts}),
    x="fratio",
    y="count",
    log_x=True,
    width=600,
    height=200,
)
fig.update_xaxes(range=[np.log10(0.6), np.log10(2.6)], type="log")
fig.update_xaxes(minor=dict(showgrid=True, nticks=10))
fig.update_traces(
    hovertemplate="frequency(ratio wrt root): %{x}<br>occurence: %{y}"
)
fig.update_layout(
    margin=dict(l=10, r=20, t=30, b=10),
)

fig.show()

In [ ]:
# create a waveform
y_from_p_sample = []
for anno in p_sample:
    tone = librosa.tone(
        2 * anno, sr=sr, length=sr / ar
    )
    y_from_p_sample += tone.tolist()

ipy_audio(data=y_from_p_sample, rate=sr)

## Pitch Detection Accuracy

In [ ]:
def detect_pitch(y, sr):
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr, fmin=110, fmax=440)
    # get indexes of the maximum value in each time slice
    max_indexes = np.argmax(magnitudes, axis=0)
    # get the pitches of the max indexes per time slice
    pitches = pitches[max_indexes, range(magnitudes.shape[1])]
    return pitches

In [ ]:
ARTONE = 1  # 1 pitch annotation per second --> 5 second audio
SRTONE = 22050

def create_tone(f, duration=5):
    # create a waveform
    pitch_list = [f,]*duration

    y_tone = []
    for anno in pitch_list:
        tone = librosa.tone(
            2 * anno, sr=SRTONE, length=SRTONE / ARTONE
        )
        y_tone += tone.tolist()
    
    return y_tone

In [ ]:
def create_and_detect(f0):
    y_tone = create_tone(f0)
    p0 = np.mean(detect_pitch(np.array(y_tone), SRTONE))
    return p0

In [ ]:
fs = np.arange(65, 330, 5)
ps = [create_and_detect(ff) for ff in fs]

In [ ]:
plt.figure()
plt.plot(fs, ps, label="detected pitch")
plt.plot(fs, fs, linestyle=":", label="expected pitch", c="limegreen")
plt.plot(fs, 2*fs, linestyle=":", label="expected pitch", c="limegreen")
plt.legend()

In [ ]:
plt.plot(fs[:30], ps[:30]-2*fs[:30], linestyle=":", label="delta (detected - expected) pitch")
plt.ylabel("delta (Hz)")
plt.xlabel("true pitch (Hz)")
plt.legend()